In [5]:
import pandas as pd


# This script performs data cleaning and merging for supplier data.
# It standardizes column names, handles missing values, and merges two datasets.
# Ensure all files are in the expected format before running.

# Load the datasets
file_supplier1 = "/content/sample_data/supplier_data_1.xlsx"
file_supplier2 = "/content/sample_data/supplier_data_2.xlsx"

# Read Excel files
df_supplier1 = pd.read_excel(file_supplier1, sheet_name="Sheet1")
df_supplier2 = pd.read_excel(file_supplier2, sheet_name="Sheet1")

# Step 1: Standardizing column names
column_mapping_supplier1 = {
    "Werksgüte": "MATERIAL_QUALITY_NORM",
    "Bestellgütentext": "MATERIAL_NAME",
    "Nenndicke NNN.NN mm mit Dezimalpunkt": "NOMINAL_THICKNESS_MM",
    "Breite": "WIDTH_MM",
    "Länge": "LENGTH_MM",
    "Gewicht (kg)": "MASS_MIN_KG",
    "Cluster": "SITE"
}

column_mapping_supplier2 = {
    "Material": "MATERIAL_NAME",
    "Article ID": "MATERIAL_NUMBER",
    "Weight (kg)": "MASS_MIN_KG",
    "Quantity": "QUANTITY",
    "Reserved": "RESERVED",
}

# Rename columns to unify both datasets
df_supplier1.rename(columns=column_mapping_supplier1, inplace=True)
df_supplier2.rename(columns=column_mapping_supplier2, inplace=True)



In [6]:
# Step 2: Convert numeric values to correct format
numeric_columns = ["NOMINAL_THICKNESS_MM", "WIDTH_MM", "LENGTH_MM", "MASS_MIN_KG"]
for col in numeric_columns:
    if col in df_supplier1.columns:
        df_supplier1[col] = pd.to_numeric(df_supplier1[col], errors='coerce')
    if col in df_supplier2.columns:
        df_supplier2[col] = pd.to_numeric(df_supplier2[col], errors='coerce')

# Step 3: Handle missing values
df_supplier1.fillna({"MATERIAL_QUALITY_NORM": "Unknown", "MATERIAL_NAME": "Unknown", "SITE": "Unknown"}, inplace=True)
df_supplier2.fillna({"MATERIAL_NAME": "Unknown", "RESERVED": "No"}, inplace=True)

# Normalize text formatting for categorical columns
categorical_columns = ["MATERIAL_QUALITY_NORM", "MATERIAL_NAME", "SITE"]
for col in categorical_columns:
    if col in df_supplier1.columns:
        df_supplier1[col] = df_supplier1[col].astype(str).str.strip().str.lower()
    if col in df_supplier2.columns:
        df_supplier2[col] = df_supplier2[col].astype(str).str.strip().str.lower()



In [9]:
# Step 4: Merge datasets on MATERIAL_NAME
merged_inventory = pd.merge(df_supplier1, df_supplier2, on="MATERIAL_NAME", how="outer")
merged_inventory.dropna(axis=1, how='all', inplace=True)
merged_inventory.to_csv("cleaned_inventory_dataset.csv", index=False)
merged_inventory.head()

,MATERIAL_QUALITY_NORM_x,MATERIAL_NAME,NOMINAL_THICKNESS_MM_x,WIDTH_MM_x,LENGTH_MM_x,MASS_MIN_KG_x,SITE_x,Si-Gehalt,Mn-Gehalt,P-Gehalt,...,MATERIAL_QUALITY_NORM_y,DEFECT_NOTES,NOMINAL_THICKNESS_MM_y,WIDTH_MM_y,LENGTH_MM_y,MASS_MIN_KG_y,INCO_TERM,BUY_NOW_EUR_PER_TON,MIN/MAX_BID_EUR_PER_TON,VALID_UNTIL
0,g2ub5,2a lager,247.0,NaN,450.0,11.76,elo verzinkt,10,A,610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,unknown,2a lager,246.0,NaN,462.0,12.04,elo verzinkt,10,A,610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,g2ub5,2a lager,247.0,NaN,873.0,22.90,elo verzinkt,10,A,610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,g2ub5,2a lager,247.0,NaN,460.0,12.05,elo verzinkt,10,A,610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c4lf5,42crmo4,302.0,NaN,809.0,22.68,unknown,0.1640,0.6700,0.0080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
